In [4]:
sdss_general_path = "C:\\Users\\lrik\\Work\\"
sdss_filename_fits = "specObj-dr17.fits"
sdss_filename_origin = f"{sdss_general_path}sdssdr17.csv"
sdss_filename_class_sample = f"{sdss_general_path}SDSS.csv"

In [3]:
import wget
sdssdr17_url = "https://data.sdss.org/sas/dr17/sdss/spectro/redux/specObj-dr17.fits"
wget.download(sdssdr17_url, 'specObj-dr17.fits')

'C:\\Users\\lrik\\Work\\specObj-dr17.fits'

In [1]:
import pandas as pd
import numpy as np
import os

from network import NN
from data_process import data_preparation, get_features

In [31]:
from astropy.io import fits
fits_file = fits.open(sdss_filename_fits)
hdr = fits_file[1].header
#['SURVEY' 'PLATEQUALITY' 'SOURCETYPE' 'PLUG_RA' 'PLUG_DEC' 'CLASS' 'SUBCLASS' 'Z' 'Z_ERR' 'ZWARNING' 'Z_NOQSO' 'Z_ERR_NOQSO' 'ZWARNING_NOQSO' 'CLASS_NOQSO' 'SUBCLASS_NOQSO']
columns = [hdr[14],hdr[19],hdr[50],hdr[80],hdr[81],hdr[82],hdr[83],hdr[84],hdr[85],hdr[103],hdr[135],hdr[136],hdr[137],hdr[138],hdr[139]]
#ax1_count = hdr[3]
ax2_count = hdr[4] #count
data = fits_file[1].data
f = open(sdss_filename_origin,"w")
f.write(','.join(columns) + "\n")
for i in range(int(ax2_count)):
	f.write(f"{str(data[i][0])},{str(data[i][5])},{str(data[i][36])},{str(data[i][66])},{str(data[i][67])},{str(data[i][68])},{str(data[i][69])},{str(data[i][70])},{str(data[i][71])},{str(data[i][89])},{str(data[i][121])},{str(data[i][122])},{str(data[i][123])},{str(data[i][124])},{str(data[i][125])}\n")
f.close()
fits_file.close()

del data, f, fits_file, i, columns, ax2_count #ax1_count

In [105]:
data = pd.read_csv(sdss_filename_origin, header=0, sep=",")

C:\Users\lrik\AppData\Local\Temp\ipykernel_21676\3876214978.py:1: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(sdss_filename_origin, header=0, sep=",")


<h1>PreClassification<h1>

In [126]:
data = data[data["ZWARNING"] == 0]

In [136]:
class_name_column = "class_name"

data.loc[(data["CLASS"] == "QSO") & ((data["SUBCLASS"].str.contains("STAR", na=True)) | (data["SUBCLASS"] == "BROADLINE")), class_name_column] = "qso"
#
data.loc[((data["CLASS"] == "QSO") | (data["CLASS"] == "GALAXY")) & (data["SUBCLASS"].str.contains("AGN", na=False)), class_name_column] = "agn"
#
data.loc[(data["CLASS"] == "GALAXY") & (data["SUBCLASS"].str.contains("STARBURST", na=False)), class_name_column] = "sbg"
#
data.loc[(data["CLASS"] == "GALAXY") & (data["SUBCLASS"].str.contains("STARFORMING", na=False)), class_name_column] = "sfg"
#
data.loc[(data["CLASS"] == "GALAXY") & ((data["SUBCLASS"].str.contains("NaN",na=True) | (data["SUBCLASS"] == "BROADLINE"))), class_name_column] = "gal"
#
data.loc[data["CLASS"] == "STAR", class_name_column] = "star"

del class_name_column


In [141]:
data = data.rename(columns={"PLUG_RA": "RA", "PLUG_DEC": "DEC", "Z": "z", "Z_ERR": "z_err"})

In [155]:
data = data.sort_values(by=["DEC"], ascending=True)
data

,SURVEY,PLATEQUALITY,SOURCETYPE,RA,DEC,CLASS,SUBCLASS,z,z_err,ZWARNING,Z_NOQSO,Z_ERR_NOQSO,ZWARNING_NOQSO,CLASS_NOQSO,SUBCLASS_NOQSO,class_name
1609086,segue1,good,NONLEGACY,7.715366,-19.702714,STAR,F5,0.000248,0.000022,0,0.0,0.0,0,NaN,NaN,star
1609168,segue1,good,NONLEGACY,7.487451,-19.669474,STAR,F9,0.000093,0.000010,0,0.0,0.0,0,NaN,NaN,star
1609200,segue1,good,NONLEGACY,7.259242,-19.656897,STAR,M0,0.000038,0.000008,0,0.0,0.0,0,NaN,NaN,star
1609083,segue1,good,NONLEGACY,7.679973,-19.641483,STAR,F5,-0.000613,0.000022,0,0.0,0.0,0,NaN,NaN,star
1609048,segue1,good,NONLEGACY,8.139904,-19.639879,STAR,G2,-0.000164,0.000014,0,0.0,0.0,0,NaN,NaN,star
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451450,segue1,good,NONLEGACY,98.019854,84.791898,STAR,F5,-0.000761,0.000009,0,0.0,0.0,0,NaN,NaN,star
1455880,segue1,good,NONLEGACY,97.666877,84.793093,STAR,F2,-0.000391,0.000023,0,0.0,0.0,0,NaN,NaN,star
1451418,segue1,good,NONLEGACY,92.883648,84.794836,STAR,F9,-0.000158,0.000009,0,0.0,0.0,0,NaN,NaN,star
1455809,segue1,good,NONLEGACY,94.478598,84.837773,STAR,F9,-0.000321,0.000017,0,0.0,0.0,0,NaN,NaN,star


In [156]:
data.to_csv(sdss_filename_class_sample, index=False)

<h1>MAIN</h1>

In [2]:
import json

class Config():

    def __init__(self,fconfig):
        config = json.load(fconfig)
        self.name_main_sample = config['name_sample']
        self.name_sample = config['name_sample'] + config['additional_name']
        self.general_path = config['general_path']
        self.data_path = config['data_path']
        self.prediction_path = config['prediction_path']
        self.test_path = config['test_path'] ############
        self.flags = config['flags']
        self.hyperparam = config['hyperparam']
        self.features = config['features']
        self.name_class = config['name_class']
        self.name_class_column = config["name_class_column"]
        self.base = config["base"]
        self.picture = config['picture']
        self.statistic = config['statistic']
        #
        self.path_ml_data = f'{self.general_path}/ml/data'
        self.path_ml = f'{self.general_path}/ml'
        self.path_sample = f'{self.general_path}/sample'
        self.path_pic = f'{self.general_path}/ml/picture'
        self.path_stat = f'{self.general_path}/statistic'
        #
        self.path_model = f"{self.path_ml}/model/mod_{self.name_sample}"
        self.path_weight = f"{self.path_ml}/model/weight_{self.name_sample}"
        self.path_eval = f"{self.path_ml}/eval/{self.name_sample}"
        self.path_predict = f"{self.path_ml}/prediction/{self.name_sample}"
        #
        columns = []
        columns_prob = []
        for col in self.name_class:
            columns.append(f"{col}_cls")
            columns_prob.append(f"{col}_cls_prob")
        self.name_class_cls = columns
        self.name_class_prob = columns_prob

<h1>Read config<h1>

In [3]:
fconfig = open('config_agn_sdss.json')
config = Config(fconfig)
fconfig.close()
del fconfig

<h1>Create file system<h1>

In [5]:
def dir(save_path,name):
    dir_name = f"{save_path}/{name}"
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)

if not os.path.isdir(config.general_path):
    os.mkdir(config.general_path)

dir(config.general_path,'sample')
dir(config.general_path,'statistic')
dir(config.general_path,'ml')
dir(config.path_ml,'data')
dir(config.path_ml,'model')
dir(config.path_ml,'eval')
dir(config.path_ml,'prediction')
dir(config.path_ml,'picture')

<h1>Downloading&Cross-match<h1>

In [13]:
from data_download import Download
config_base = dict(name_class=config.name_class,
                   base=config.base,
                   features=config.features["data"],
                   path_sample=config.path_sample)
#print(config_base["features"])
dow = Download(config_spec=config.flags["data_downloading"],config_base=config_base)

In [157]:
if(config.flags["data_downloading"]["class_diff"]):
    dow.diff_class(config.data_path,config.name_class_column)

qso 		 961543.0
agn 		 36178.0
sbg 		 99393.0
sfg 		 287815.0
gal 		 2447433.0
star 		 1022629.0


In [8]:
if(config.flags["data_downloading"]["work"]):
    dow.all_class_download(config.path_stat)

del dow

cut to slice start for  C:\Users\lrik\Work\agn_sdss/sample/qso_origin.csv
Origin catalog columns:	 ['RA', 'DEC', 'z', 'z_err']
cut to slice finish with  9  slices
total count of lines is  961543
C:\Users\lrik\Work\agn_sdss/sample/qso_origin/0_catwise.csv
{'astrometry': ['angDist', 'PQSO', 'PGal', 'PSS'], 'flux': [], 'photometry': ['W1mproPM', 'e_W1mproPM', 'W2mproPM', 'e_W2mproPM', 'Gmag', 'e_Gmag', 'BPmag', 'e_BPmag', 'RPmag', 'e_RPmag']}
[1, 2, 3, 4, 0, 35, 36, 37, 38]

C:\Users\lrik\Work\agn_sdss/sample/qso_origin/1_catwise.csv
{'astrometry': ['angDist', 'PQSO', 'PGal', 'PSS'], 'flux': [], 'photometry': ['W1mproPM', 'e_W1mproPM', 'W2mproPM', 'e_W2mproPM', 'Gmag', 'e_Gmag', 'BPmag', 'e_BPmag', 'RPmag', 'e_RPmag']}
[1, 2, 3, 4, 0, 35, 36, 37, 38]

C:\Users\lrik\Work\agn_sdss/sample/qso_origin/2_catwise.csv
{'astrometry': ['angDist', 'PQSO', 'PGal', 'PSS'], 'flux': [], 'photometry': ['W1mproPM', 'e_W1mproPM', 'W2mproPM', 'e_W2mproPM', 'Gmag', 'e_Gmag', 'BPmag', 'e_BPmag', 'RPmag', 'e_R

<h1>Preprocessing pipeline<h1>

In [4]:
data = pd.DataFrame()
#data preparation
if(not config.flags['data_preprocessing']['work']):
    if(os.path.isfile(f'{config.path_ml_data}/{config.name_main_sample}_all.csv')):
        data = pd.read_csv(f'{config.path_ml_data}/{config.name_main_sample}_all.csv', header = 0, sep = ',')
    else:
        data = data_preparation(config.path_ml_data,config.path_sample,config.name_class,config)
else:
    data = data_preparation(config.path_ml_data,config.path_sample,config.name_class,config)

read data qso
                RA        DEC         z     z_err   angDist  angDist.1  \
0        24.209379  -9.652945  0.424225  0.000034  0.115775   0.086731   
1         2.705408  -9.651789  1.501741  0.000450  0.155692   0.129829   
2         2.750980  -9.651008  1.569697  0.001337  0.144322   0.511774   
3        28.026975  -9.650310  1.228188  0.000341  0.162832   0.226970   
4        84.556206  -9.650050  5.900047  0.000756  0.184939   0.166734   
...            ...        ...       ...       ...       ...        ...   
391418  188.809740  66.985638  1.478909  0.000589  0.275694   0.483635   
391419  191.804300  66.988517  0.722520  0.000287  0.174089   0.271125   
391420  208.131240  66.989877  2.019768  0.000580  0.284380   0.683047   
391421  182.505910  66.994894  1.107985  0.000295  0.084471   0.060162   
391422  202.142030  66.996430  0.813712  0.000085  0.176417   0.087861   

            PQSO      PGal       PSS  W1mproPM  e_W1mproPM  W2mproPM  \
0       0.999696  0.00030

ModuleNotFoundError: No module named 'extinction_coefficient'

In [14]:
print(data)

        index          RA        DEC         z     z_err      PM     Plx  \
0      128185   36.051854   2.158975  0.000193  0.000014  11.023  0.0359   
1       77411  246.576140  50.067786  0.190841  0.000030   0.000  0.0000   
2       21822  184.426730   9.843449  0.094431  0.000009   0.900  3.7952   
3       77155  330.113260   4.682897  1.646827  0.000365   1.098 -0.6689   
4       81545  129.145120  56.791925  0.220036  0.000010   0.000  0.0000   
...       ...         ...        ...       ...       ...     ...     ...   
10138  242081  357.650810  29.812984  1.535003  0.000532   2.607  2.4911   
10139   23846  172.312890  62.991410  0.043615  0.000005   0.000  0.0000   
10140   11858  113.953460  40.327300  0.042130  0.000009   0.000  0.0000   
10141  141503  217.495700  14.863021  0.095066  0.000028   0.000  0.0000   
10142   14047  234.348160  49.694834  0.118916  0.000017   1.016 -0.1655   

        e_Plx   RUWE   pscol  ...     mcd_g  fuzzy_err  fuzzy_dist  QSO_cls  \
0      0

Basic statistic from sample

In [15]:
#data_statistic
if(config.statistic["metric"]):
    data.describe().transpose().to_csv(f'{config.path_stat}/{config.name_main_sample}_stat.log')

In [ ]:
#network training
if(config.hyperparam["model_variable"]["work"]):
    #features from config
    #name from config
    print('Sample name: ', config.name_sample)
    print('Features: ', config.features["data"])
    #hyperparams from config
    batch_size = config.hyperparam['batch_size']
    num_ep = config.hyperparam['num_ep']
    optimizer = config.hyperparam['optimizer']
    loss = config.hyperparam['loss']
    validation_split = config.hyperparam['validation_split']
    #balanced class
    class_weights = None
    if(config.hyperparam["model_variable"]["balanced"]):
        from sklearn.utils import class_weight
        y = np.zeros(data.shape[0])
        cl = np.array(data[config.name_class_cls].values)
        #print(cl)
        for i in range(data.shape[0]):
            y[i] = np.argmax(cl[i,:])
        class_weights = dict(enumerate(class_weight.compute_class_weight(class_weight = 'balanced',classes = np.unique(y),y = y)))
        print("class weights",class_weights)
        del y

    print(data)

    print("Features mode list:\t",config.features["train"])
    features = get_features(config.features["train"],config)
    print("Features train values:\t",features)
    
    sample_weight = None
    if(config.hyperparam["model_variable"]["sample_weight"] in config.flags['data_preprocessing']['main_sample']['weight']['method']):
        #sample_weight = data[config.hyperparam["model_variable"]["sample_weight"]].values
        sample_weight = data[config.hyperparam["model_variable"]["sample_weight"]].values.T[0]
        print(sample_weight)

    try:
        data[features]
    except:
        raise Exception("data don't have initiated features, check config.features['train'] value and WARNINGs above")

    if(config.hyperparam["model_variable"]["work"]):
        NN(data[features],data[config.name_class_cls],data['z'],sample_weight,validation_split,batch_size,num_ep,optimizer,loss,class_weights,
        output_path_predict = config.path_predict,
        output_path_mod = config.path_model,
        output_path_weight = config.path_weight,
        path_save_eval = config.path_eval,
        config=config)

<h1>Metric statistic<h1>

In [ ]:
#statistic
from statistic import metric_statistic
if(config.statistic["metric"]):
    metric_statistic(config)

In [ ]:
print(data)

In [1]:
from graphic import picture_confusion_matrix, picture_roc_prc, picture_hist, TSNE_pic, contam_dist_pic, multigridplot, picture_correlation_matrix


In [ ]:
picture_correlation_matrix(data[get_features(config.features["train"],config)],"mags",config)

In [ ]:
TSNE_pic(data,config)

In [ ]:
contam_dist_pic(data,config)

In [ ]:
multigridplot(data,get_features(config.features["train"],config),config)

In [ ]:
picture_roc_prc(config)

In [ ]:
picture_confusion_matrix(config)

In [ ]:
picture_hist(data,config)

In [ ]:
from graphic import redshift_estimation

redshift_estimation(config)